# Set up OpenAI API

In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()  # This loads the variables from .env into the environment


True

In [3]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI()

In [4]:
import requests
import json

# URL of the JSON data
url = 'https://gezhangrp.com/research-data/RunExp/data/ac17_6_9-May-09-2023.json'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON content
    data = response.json()
    # Now `data` is a Python dictionary containing the JSON data
    # print(data)
else:
    print(f"Failed to retrieve data: HTTP {response.status_code}")

# If you need to handle more complex JSON structures or errors, you can use
# json.loads(response.text) instead of response.json() for more control.


In [5]:
import random

In [6]:
id_list = data['code'].keys()

In [7]:
correct_id_list = [i for i in id_list if data['test'][i]]

In [8]:
print(len(id_list), len(correct_id_list))

3942 1464


In [9]:
sampled_id_list = random.sample(correct_id_list, 100)

In [10]:
len(sampled_id_list)

100

In [11]:
sampled_code_list = [data['code'][i] for i in sampled_id_list]

In [12]:
random.choice(list(data['code'].values()))

"t = []\nother = []\nathletes = [['Phelps', 'Lochte', 'Schooling', 'Ledecky', 'Franklin'], ['Felix', 'Bolt', 'Gardner', 'Eaton'], ['Biles', 'Douglas', 'Hamm', 'Raisman', 'Mikulak', 'Dalton']]\n#print(athletes)\nfor ls in athletes:\n    for wd in ls:\n        print(wd)\n        \n            "

In [13]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [14]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [26]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "find_code_line_and_step",
            "description": "Find code line k in a code solution and its step",
            "parameters": {
                "type": "object",
                "properties": {

                    "step": {
                        "type": "string",
                        "enum": all_steps,
                        "description": "The step of this code line",
                    },
                    "code": {
                        "type": "string",
                        # "enum": all_steps,
                        "description": "The code of this code line",
                    },
                },
                "required": ["step"]
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_steps_in_programming_exercise",
            "description": "Given a programming exercise, give high levels steps to solve the exercise",
            "parameters": {
                "type": "object",
                "properties": {
                    "step1": {
                        "type": "string",
                        "description": "The 1st step",
                    },
                    "step2": {
                        "type": "string",
                        # "enum": ["celsius", "fahrenheit"],
                        "description": "The 2nd step",
                    },
                    "step3": {
                        "type": "string",
                        "description": "The 3rd step",
                    },
                    "step4": {
                        "type": "string",
                        "description": "The 4th step",
                    },
                    "step5": {
                        "type": "string",
                        "description": "The 5th step",
                    },
                    "step6": {
                        "type": "string",
                        "description": "The 6th step",
                    },
                    "step7": {
                        "type": "string",
                        "description": "The 7th step",
                    },
                    "step8": {
                        "type": "string",
                        "description": "The 8th step",
                    },
                    "step9": {
                        "type": "string",
                        "description": "The 9th step",
                    },
                    "step10": {
                        "type": "string",
                        "description": "The 10th step",
                    }
                },
                "required": []
            },
        }
    },

    {
        "type": "function",
        "function": {
            "name": "classify_code_lines",
            "description": "Given a coding solution and a collection of steps to a programming problem, classify each code line in the solution to a step",
            "parameters": {
                "type": "object",
                "properties": {
                    "line0": {
                        "type": "string",
                        "enum": all_steps,
                        "description": "description of the step of the 0st line",
                    },
                    "line1": {
                        "type": "string",
                        # "enum": steps_dict.values(),
                        "enum": all_steps,
                        "description": "description of the step of the 1st line",
                    },
                    "line2": {
                        "type": "string",
                        # "enum": steps_dict.values(),
                        "enum": all_steps,
                        # "enum": ["celsius", "fahrenheit"],
                        "description": "The step of the 2nd line",
                    },
                    "line3": {
                        "type": "string",
                        # "enum": list(steps_dict.values()),
                        "enum": all_steps,
                        "description": "The step of the 3rd line",
                    },
                    "line4": {
                        "type": "string",
                        "enum": all_steps,
                        # "enum": list(steps_dict.values()),
                        "description": "The step of the 4th line",
                    },
                    "line5": {
                        "type": "string",
                        # "enum": list(steps_dict.values()),
                        "enum": all_steps,
                        "description": "The step of the 5th line",
                    },
                    "line6": {
                        "type": "string",
                        # "enum": list(steps_dict.values()),
                        "enum": all_steps,
                        "description": "The step of the 6th line",
                    },
                    "line7": {
                        "type": "string",
                        # "enum": list(steps_dict.values()),
                        "enum": all_steps,
                        "description": "The step of the 7th line",
                    },
                    "line8": {
                        "type": "string",
                        # "enum": list(steps_dict.values()),
                        "enum": all_steps,
                        "description": "The step of the 8th line",
                    },
                    "line9": {
                        "type": "string",
                        # "enum": list(steps_dict.values()),
                        "enum": all_steps,
                        "description": "The step of the 9th line",
                    },
                    "line10": {
                        "type": "string",
                        "enum": all_steps,
                        # "enum": list(steps_dict.values()),
                        "description": "The step of the 10th line",
                    }
                },
                "required": []
            },
        }
    },
    
]

In [19]:
# segment a code solution into steps
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "This is a programming exercise. I will provide the description in next message."})
messages.append({"role": "user", "content": ''' `athletes` is a nested list of strings. Create a list, ``t``, that saves only the athlete's name if it contains the letter "t". If it does not contain the letter "t", save the athlete name into list ``other``.'''})
messages.append({"role": "user", "content": "What are the high level steps to solve the problem?"})

chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_steps_in_programming_exercise"}}
)
chat_response.choices[0]


Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fYTume3TTCLp4KEfnRP6jqR4', function=Function(arguments='{\n  "step1": "Create an empty list named \'t\'",\n  "step2": "Create an empty list named \'other\'",\n  "step3": "Iterate over each athlete in the \'athletes\' list",\n  "step4": "Check if the athlete\'s name contains the letter \'t\'",\n  "step5": "If the athlete\'s name contains \'t\', append the name to the \'t\' list",\n  "step6": "If the athlete\'s name does not contain \'t\', append the name to the \'other\' list",\n  "step7": "Repeat steps 4 to 6 for all athletes in the \'athletes\' list",\n  "step8": "Return the \'t\' and \'other\' lists"\n}', name='get_steps_in_programming_exercise'), type='function')]))

In [ ]:
[  "step1": "Create an empty list called 't'", "step2": "Create an empty list called 'other'","step3": "Iterate over each sublist in 'athletes'","step4": "Iterate over each athlete in the current sublist",  "step5": "Check if the athlete's name contains the letter 't'","step6": "If the athlete's name contains 't', append it to 't' list",  "step7": "If the athlete's name does not contain 't', append it to 'other' list"]

In [20]:
print(chat_response.choices[0].message.tool_calls[0].function.arguments)

{
  "step1": "Create an empty list named 't'",
  "step2": "Create an empty list named 'other'",
  "step3": "Iterate over each athlete in the 'athletes' list",
  "step4": "Check if the athlete's name contains the letter 't'",
  "step5": "If the athlete's name contains 't', append the name to the 't' list",
  "step6": "If the athlete's name does not contain 't', append the name to the 'other' list",
  "step7": "Repeat steps 4 to 6 for all athletes in the 'athletes' list",
  "step8": "Return the 't' and 'other' lists"
}


In [21]:
steps = chat_response.choices[0].message.tool_calls[0].function.arguments

In [22]:
steps

'{\n  "step1": "Create an empty list named \'t\'",\n  "step2": "Create an empty list named \'other\'",\n  "step3": "Iterate over each athlete in the \'athletes\' list",\n  "step4": "Check if the athlete\'s name contains the letter \'t\'",\n  "step5": "If the athlete\'s name contains \'t\', append the name to the \'t\' list",\n  "step6": "If the athlete\'s name does not contain \'t\', append the name to the \'other\' list",\n  "step7": "Repeat steps 4 to 6 for all athletes in the \'athletes\' list",\n  "step8": "Return the \'t\' and \'other\' lists"\n}'

In [45]:
def classify_line(k, code):
    messages = []
    messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
    messages.append({"role": "user", "content": "This is a programming exercise. I will provide the description in next message."})
    messages.append({"role": "user", "content": ''' `athletes` is a nested list of strings. Create a list, ``t``, that saves only the athlete's name if it contains the letter "t". If it does not contain the letter "t", save the athlete name into list ``other``.'''})
    messages.append({"role": "user", "content": "To solve the problem, there are several steps it takes: {}. ".format(steps)})
    
    # code = random.choice(sampled_code_list)
    # code = '\n'.join([l for l in code.split('\n') if l.strip()])
    
    messages.append({"role": "user", "content": "For the following code solution, code line {} is `{}`. Which step it is? ".format(k, code.split('\n')[k-1])})
    
    
    messages.append({"role": "user", "content": code})
    
    chat_response = chat_completion_request(
        messages, tools=tools, tool_choice={"type": "function", "function": {"name": "find_code_line_and_step"}}
        # messages, tools=tools, tool_choice="none"
    )
    # print(chat_response.choices[0].message.tool_calls[0].function.arguments)
    return json.loads(chat_response.choices[0].message.tool_calls[0].function.arguments)

In [44]:
code

"\nathletes = [['Phelps', 'Lochte', 'Schooling', 'Ledecky', 'Franklin'], ['Felix', 'Bolt', 'Gardner', 'Eaton'], ['Biles', 'Douglas', 'Hamm', 'Raisman', 'Mikulak', 'Dalton']]\n\nt = []\nother = []\n\nfor lst in athletes:\n    for name in lst:\n        if 't' in name:\n            t.append(name)\n        else:\n            other.append(name)\n\n"

In [43]:
code = sampled_code_list[56]
lines = code.split('\n')
code_steps = []
for i, l in enumerate(lines):
    # print(classify_line(i+1, code))
    code_steps.append(classify_line(i+1, code))

{
  "step": "Create an empty list named 't'",
  "code": "t = []"
}
{
  "step": "Create an empty list named 't'",
  "code": "t = []"
}
{
  "step": "Iterate over each athlete in the 'athletes' list",
  "code": "for lst in athletes:"
}
{
  "step": "Create an empty list named 't'",
  "code": "t = []"
}
{
  "step": "Create an empty list named 'other'",
  "code": "other = []"
}
{
  "step": "If the athlete's name does not contain 't', append the name to the 'other' list",
  "code": "other.append(name)"
}
{
  "step": "Iterate over each athlete in the 'athletes' list",
  "code": "for lst in athletes:"
}
{
  "step": "Iterate over each athlete in the 'athletes' list",
  "code": "    for name in lst:"
}
{
  "step": "Check if the athlete's name contains the letter 't'",
  "code": "        if 't' in name:"
}
{
  "step": "If the athlete's name contains 't', append the name to the 't' list",
  "code": "            t.append(name)"
}
{
  "step": "",
  "code": "        else:"
}
{
  "step": "If the athlet

In [32]:
code = random.choice(sampled_code_list)
code = '\n'.join([l for l in code.split('\n') if l.strip()])

In [37]:
def get_results(code):
    lines = code.split('\n')
    code_steps = []
    for i, l in enumerate(lines):
        # print(classify_line(i+1, code))
        code_steps.append(classify_line(i+1, code))
        # print('-'*20)
    # print(code_steps)
    return code_steps

In [38]:
from tqdm import tqdm

In [39]:
for code in tqdm(sampled_code_list):
    t = 1

100%|██████████████████████████████████| 100/100 [00:00<00:00, 104700.55it/s]


In [46]:
# all_results = []
for code in tqdm(sampled_code_list[56:]):
    all_results.append(get_results(code))    

100%|████████████████████████████████████████| 44/44 [11:12<00:00, 15.29s/it]


In [47]:
len(all_results)

100

In [48]:
!pwd

/Users/gezhang/Research/src/Map-Pro/notebooks


In [49]:
with open('../data/steps.json', 'w') as fout:
    json.dump(all_results, fout)

In [28]:
results = chat_response.choices[0].message.tool_calls[0].function.arguments

In [266]:
results = chat_response.choices[0].message.content

In [29]:
results = json.loads(results)

In [30]:
print(results)

{'step': "Iterate over each athlete in the 'athletes' list", 'code': 'for athlete in athletes:'}


In [31]:
print(code)

t=[]
other=[]
athletes = [['Phelps', 'Lochte', 'Schooling', 'Ledecky', 'Franklin'], ['Felix', 'Bolt', 'Gardner', 'Eaton'], ['Biles', 'Douglas', 'Hamm', 'Raisman', 'Mikulak', 'Dalton']]
for athlete in athletes:
    for name in athlete:
        if 't' in name:
            t.append(name)
        else:
            other.append(name)


In [244]:
messages = []
messages.append({"role": "user", "content": "you said there is 7 code lines but gave 9. Why"})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
print(chat_response.choices[0].message.content)

I apologize for the confusion. It seems there was a mistake in my response. You mentioned that there are 7 code lines, but I provided 9 options for each line. Please disregard the extra options and focus on the 7 code lines you mentioned. Thank you for bringing this to my attention, and I apologize for any inconvenience caused.


In [23]:
steps_dict = json.loads(steps)

In [24]:
steps_dict.values()

dict_values(["Create an empty list named 't'", "Create an empty list named 'other'", "Iterate over each athlete in the 'athletes' list", "Check if the athlete's name contains the letter 't'", "If the athlete's name contains 't', append the name to the 't' list", "If the athlete's name does not contain 't', append the name to the 'other' list", "Repeat steps 4 to 6 for all athletes in the 'athletes' list", "Return the 't' and 'other' lists"])

In [25]:
all_steps = list(steps_dict.values())

In [201]:
all_steps = ["Create an empty list called 't'", "Create an empty list called 'other'", "Iterate over each sublist in 'athletes'", 'Iterate over each athlete in the current sublist', "Check if the athlete's name contains the letter 't'", "If the athlete's name contains 't', append it to 't' list", "If the athlete's name does not contain 't', append it to 'other' list"]

In [184]:
lines = code.split('\n')
for i, line in enumerate(lines):
    # print(results["line{}".format(i+1)])
    print(steps_dict[results["line{}".format(i+1)]])
    print(line)
    print('-'*20)

KeyError: "Create an empty list called 't'"

In [125]:
results

{'line1': 'step1',
 'line2': 'step2',
 'line3': 'step3',
 'line4': 'step4',
 'line5': 'step5',
 'line6': 'step6',
 'line7': 'step7'}

In [127]:
print(code)

athletes = [['Phelps', 'Lochte', 'Schooling', 'Ledecky', 'Franklin'], ['Felix', 'Bolt', 'Gardner', 'Eaton'], ['Biles', 'Douglas', 'Hamm', 'Raisman', 'Mikulak', 'Dalton']]
t = []
other = []
for athlete_l in athletes:
    for athlete_name in athlete_l:
        if 't' in athlete_name:
            t.append(athlete_name)
        else:
            other.append(athlete_name)


In [106]:
print(chat_response.choices[0].message.tool_calls[0].function.arguments)

{
  "line1": "step1",
  "line2": "step2",
  "line3": "step3",
  "line4": "step3",
  "line5": "step5",
  "line6": "step6",
  "line7": "step7",
  "line8": "step6",
  "line9": "step7",
  "line10": "step9"
}
